만든날짜: 0414  
stellar_env 가상환경 사용. 파이썬 3.8버전  
stellargraph를 불러오려면 chardet이라는 라이브러리가 별도로 필요함 -> 텐서플로우, 스텔라그래프, chardet 모두 설치  
안됨 3.10버전에서 텐서플로우 2.9버전, 스텔라그래프 콘다 채널로 설치 + chardet (sage_env 가상환경)

In [ ]:
# --- StellarGraph 및 TensorFlow 임포트 ---

try:
    import stellargraph as sg
    from stellargraph.mapper import GraphSAGENodeGenerator # GraphSAGENodeGenerator 임포트
    from stellargraph.layer import GraphSAGE
    print(f"StellarGraph version: {sg.__version__}")
except ImportError:
    print("StellarGraph 또는 관련 모듈을 찾을 수 없습니다. 설치를 확인하세요.")
    exit()

try:
    import tensorflow as tf
    from tensorflow.keras import layers, optimizers, losses, metrics, Model
    print(f"TensorFlow version: {tf.__version__}")
except ImportError:
    print("TensorFlow를 찾을 수 없습니다. 설치를 확인하세요.")
    exit()

# (다른 필요한 라이브러리 import 계속...)
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelBinarizer, StandardScaler
from sklearn.feature_extraction.text import TfidfVectorizer
import io # 임시 데이터용

c:\Users\eric0\AppData\Local\anaconda3\envs\sage_env\lib\site-packages\h5py\__init__.py:36: UserWarning: h5py is running against HDF5 1.14.6 when it was built against 1.14.5, this may cause problems
  _warn(("h5py is running against HDF5 {0} when it was built against {1}, "


StellarGraph version: 1.2.1
TensorFlow version: 2.9.1


In [2]:
import sys
print(f"Python Executable: {sys.executable}") # 현재 커널의 파이썬 경로 확인
print(f"Python Version: {sys.version}")

try:
    import tensorflow as tf
    print(f"TensorFlow Version: {tf.__version__}") # TF 임포트 및 버전 확인
except Exception as e:
    print(f"TensorFlow 임포트 오류: {e}")

try:
    import stellargraph as sg
    print(f"StellarGraph Version: {sg.__version__}") # StellarGraph 임포트 및 버전 확인
except Exception as e:
    print(f"StellarGraph 임포트 오류: {e}")

print("임포트 테스트 완료")

Python Executable: c:\Users\eric0\AppData\Local\anaconda3\envs\sage_env\python.exe
Python Version: 3.10.13 | packaged by Anaconda, Inc. | (main, Sep 11 2023, 13:15:57) [MSC v.1916 64 bit (AMD64)]
TensorFlow Version: 2.9.1
StellarGraph Version: 1.2.1
임포트 테스트 완료


In [3]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelBinarizer, StandardScaler
from sklearn.feature_extraction.text import TfidfVectorizer

In [4]:
edge_df = pd.read_csv("C:/Project/RAG연구동향분석/250409/전체기간(1-7)/전체기간(1-7)엣지.csv")

In [5]:
import pandas as pd
import io

# --- 1. 데이터 로딩 ---
try:
    # 기존 노드 데이터 로드 (DtypeWarning 처리 위해 low_memory=False 추가)
    node_df = pd.read_csv("C:/Project/RAG연구동향분석/250409/전체기간(1-7)/전체기간(1-7)노드.csv", low_memory=False)
    print("기존 node_df 로드 완료.")
    print("로드 전 node_df info:")
    # node_df.info(memory_usage='deep') # 필요시에만 info() 호출

    # 전체 정보가 담긴 Parquet 파일 로드
    parquet_path = 'C:/Project/RAG연구동향분석/데이터/dblp_v14_filtered.parquet'

    # **** 중요: Parquet 파일의 실제 컬럼 이름과 대소문자까지 정확히 일치시키세요 ****
    # 오류 메시지의 스키마 정보를 참고하여 수정 (예: Id -> id, Label -> title)
    columns_to_read = [
        'id',          # Parquet 파일의 ID 컬럼명 (소문자 'i')
        'title',       # Parquet 파일의 제목 컬럼명 (Label 대신 title?)
        'year',        # Parquet 파일의 연도 컬럼명
        'n_citation',  # Parquet 파일의 인용수 컬럼명 (citations 대신 n_citation?)
        'venue',       # Parquet 파일의 venue 컬럼명 (추가할 경우)
        'keywords',    # Parquet 파일의 키워드 컬럼명 (list 형태일 수 있음)
        'fos.name',    # Parquet 파일의 연구분야 컬럼명 (research_fields 대신 fos.name?)
        # 'level' 컬럼은 Parquet 스키마에 없어 보이므로 제외, 필요시 확인/추가
    ]
    source_df = pd.read_parquet(parquet_path, columns=columns_to_read)
    print(f"\nParquet 파일 '{parquet_path}' 로드 완료.")
    print("로드된 source_df info:")
    # source_df.info(memory_usage='deep') # 필요시에만 info() 호출

    # **** Parquet 데이터 컬럼 이름 변경 (node_df와 맞추기 위해, 필요시) ****
    # 예: source_df의 'id'를 'Id'로, 'title'을 'Label'로 변경 등
    source_df.rename(columns={
        'id': 'Id',
        'title': 'Label',
        'n_citation': 'citations',
        'fos.name': 'research_fields',
        # 'venue': 'venue_from_source', # 이름 충돌 방지 예시
        # 'level'은 source_df에 없으므로 rename 대상 아님
    }, inplace=True)
    print("\nsource_df 컬럼 이름 변경 완료 (node_df와 맞추기 위해).")
    print(f"변경된 source_df 컬럼: {source_df.columns.tolist()}")


except FileNotFoundError as e:
    print(f"파일 로딩 오류: {e}")
    print("파일 경로를 확인하세요.")
    exit()
except ImportError:
    print("오류: Parquet 파일을 읽기 위해 'pyarrow' 또는 'fastparquet' 라이브러리가 필요합니다.")
    print("pip install pandas pyarrow 또는 pip install pandas fastparquet 명령어로 설치하세요.")
    exit()
except Exception as e:
    print(f"데이터 로딩 중 오류 발생: {e}")
    exit() # 로딩 실패 시 중단

# --- 2. 데이터 전처리 (ID를 인덱스로 설정) ---
try:
    if 'Id' not in node_df.columns or 'Id' not in source_df.columns:
        print("오류: 'Id' 컬럼이 node_df 또는 source_df에 없습니다. (rename 후 확인)")
        exit()

    # ID를 인덱스로 설정 (각 DataFrame의 'Id' 컬럼 사용)
    # 원본을 유지하기 위해 새로운 변수에 할당
    node_df_indexed = node_df.set_index('Id')
    source_df_indexed = source_df.set_index('Id')

    # 인덱스 타입 통일 시도 (문제가 없다면 생략 가능)
    # node_df_indexed.index = node_df_indexed.index.astype(str)
    # source_df_indexed.index = source_df_indexed.index.astype(str)
    print("\n'Id' 컬럼을 인덱스로 설정 완료.")

except KeyError as e:
     print(f"오류: 'Id' 컬럼({e})을 인덱스로 설정하는 데 실패했습니다. 컬럼 이름을 확인하세요.")
     exit() # 인덱스 설정 실패 시 중단
except Exception as e:
     print(f"인덱스 설정 중 오류: {e}")
     exit() # 인덱스 설정 실패 시 중단


# --- 3. combine_first를 사용하여 결측값 채우기 ---
# source_df_indexed가 정상적으로 생성되었는지 확인 후 진행
if 'source_df_indexed' in locals() and 'node_df_indexed' in locals():
    print("\n결측값 채우기 시작 (combine_first)...")

    # node_df_indexed의 NaN 값을 source_df_indexed의 값으로 채움
    node_df_filled = node_df_indexed.combine_first(source_df_indexed)

    print("결측값 채우기 완료.")

    # --- 4. 결과 확인 ---
    print("\n결측값 채우기 후 node_df_filled info:")
    node_df_filled.info(memory_usage='deep')

    # (선택 사항) 인덱스를 다시 컬럼으로 되돌리기
    node_df_final = node_df_filled.reset_index()
    print("\n최종 데이터프레임 (node_df_final) info:")
    # node_df_final.info(memory_usage='deep') # 너무 길면 주석 처리

    # (선택 사항) 결측치가 얼마나 채워졌는지 확인 (예: 'year' 컬럼)
    # combine_first는 원본 컬럼을 덮어쓰므로 node_df_filled에서 확인
    original_nan_count = node_df['year'].isnull().sum() # 원본 node_df 기준
    filled_nan_count = node_df_filled['year'].isnull().sum()
    print(f"\n'year' 컬럼 결측치 변화: {original_nan_count}개 -> {filled_nan_count}개")

    # 이제 node_df_filled 또는 node_df_final을 사용하여 GraphSAGE 특징 생성 등을 진행할 수 있습니다.
    # 예: df = node_df_final # GraphSAGE 코드에서 사용할 변수명으로 할당

else:
    print("\n오류: source_df_indexed 또는 node_df_indexed가 정의되지 않아 combine_first를 실행할 수 없습니다.")

기존 node_df 로드 완료.
로드 전 node_df info:

Parquet 파일 'C:/Project/RAG연구동향분석/데이터/dblp_v14_filtered.parquet' 로드 완료.
로드된 source_df info:

source_df 컬럼 이름 변경 완료 (node_df와 맞추기 위해).
변경된 source_df 컬럼: ['Id', 'Label', 'year', 'citations', 'venue', 'keywords', 'research_fields']

'Id' 컬럼을 인덱스로 설정 완료.

결측값 채우기 시작 (combine_first)...
결측값 채우기 완료.

결측값 채우기 후 node_df_filled info:
<class 'pandas.core.frame.DataFrame'>
Index: 5259862 entries, 53a725db20f7420be8b5bfc6 to 63ac3b54d88656000c1108fb
Data columns (total 11 columns):
 #   Column            Dtype  
---  ------            -----  
 0   Degree            float64
 1   Label             object 
 2   citations         float64
 3   indegree          float64
 4   keywords          object 
 5   level             object 
 6   modularity_class  float64
 7   outdegree         float64
 8   research_fields   object 
 9   venue             object 
 10  year              float64
dtypes: float64(6), object(5)
memory usage: 3.1 GB

최종 데이터프레임 (node_df_final) info:

'

In [6]:
import pandas as pd
import os

# --- 사용자 설정: 저장할 파일 경로 ---
# 저장할 디렉토리와 파일 이름을 지정합니다. (확장자를 .csv로 변경)
output_dir_save = "C:/Project/RAG연구동향분석/250409/Preprocessed_Nodes"
output_filename_csv = "nodes_filled_final.csv" # 파일 이름 변경
output_path_csv = os.path.join(output_dir_save, output_filename_csv)

# 저장할 디렉토리가 없으면 생성
os.makedirs(output_dir_save, exist_ok=True)
# ------------------------------------

# --- 저장할 데이터프레임 확인 및 변환 ---
dataframe_to_save = None
if 'node_df_final' in locals() and isinstance(node_df_final, pd.DataFrame) and 'Id' in node_df_final.columns:
    # node_df_final은 이미 'Id' 컬럼을 가지고 있다고 가정
    dataframe_to_save = node_df_final
    print("저장 대상: node_df_final (Id 컬럼 포함)")
elif 'node_df_filled' in locals() and isinstance(node_df_filled, pd.DataFrame) and node_df_filled.index.name == 'Id':
    # node_df_filled는 'Id'가 인덱스이므로 컬럼으로 변환
    dataframe_to_save = node_df_filled.reset_index() # reset_index()로 'Id'를 컬럼으로 만듦
    print("저장 대상: node_df_filled (Id를 컬럼으로 변환하여 저장)")
else:
    print("오류: 저장할 데이터프레임('node_df_final' 또는 'node_df_filled' - Id 포함)을 찾을 수 없습니다.")
    exit() # 저장할 데이터 없으면 중단

# --- 데이터프레임 저장 ---
try:
    output_dir_save = "C:/Project/RAG연구동향분석/250409/Preprocessed_Nodes"
    output_filename = "nodes_filled_final.parquet"
    output_path = os.path.join(output_dir_save, output_filename)
    os.makedirs(output_dir_save, exist_ok=True)

    print(f"\n'{output_path}' 경로에 데이터 저장 시작...")
    # index=False 옵션으로 저장 (RangeIndex 저장 방지, 'Id'는 이미 컬럼임)
    dataframe_to_save.to_parquet(output_path, index=False)
    print(f"저장 완료: {output_path}")

except Exception as e:
    print(f"파일 저장 중 오류 발생: {e}")

저장 대상: node_df_final (Id 컬럼 포함)

'C:/Project/RAG연구동향분석/250409/Preprocessed_Nodes\nodes_filled_final.parquet' 경로에 데이터 저장 시작...
파일 저장 중 오류 발생: ('cannot mix list and non-list, non-null values', 'Conversion failed for column keywords with type object')


---
---

In [7]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelBinarizer, StandardScaler
from sklearn.feature_extraction.text import TfidfVectorizer
from collections import Counter
import re
import io # 파일 로딩 에러 시 임시 데이터용

In [8]:
df = node_df_final

# --- 1. 데이터 로딩 ---
# **** 중요: 이전 단계에서 결측치를 채운 node_df_final 변수를 사용한다고 가정 ****
# 만약 변수명이 다르거나 로딩이 필요하면 이 부분을 수정하세요.
# df = node_df_final # node_df_final 이 이미 로드되어 있다고 가정

# 엣지 데이터 로드
try:
    # 실제 파일 경로로 수정하세요
    # 만약 df = node_df_final 이 아니라면, 여기서 node_df도 로드해야 합니다.
    # 예시: node_df = pd.read_csv("C:/Project/RAG연구동향분석/250409/전체기간(1-7)/전체기간(1-7)노드.csv", low_memory=False)
    #      df = node_df # node_df_final 대신 원본 사용 시

    edges_df = pd.read_csv("C:/Project/RAG연구동향분석/250409/전체기간(1-7)/전체기간(1-7)엣지.csv")
    print("노드 및 엣지 데이터 준비 완료 (또는 로딩 완료).")

except FileNotFoundError as e:
    print(f"파일 로딩 오류: {e}")
    exit()
except Exception as e:
    print(f"데이터 로딩 중 예상치 못한 오류 발생: {e}")
    exit()

노드 및 엣지 데이터 준비 완료 (또는 로딩 완료).


In [9]:
# --- 2. 사용자 설정 변수 ---
YEAR_COLUMN = 'year'
TARGET_COLUMN = 'modularity_class'
TEXT_FEATURE_COLS = ['keywords', 'research_fields', 'Label'] # 특징으로 사용할 컬럼 (실제 df에 있는지 확인 필요)
ID_COLUMN = 'Id' # 노드 ID 컬럼 이름

In [10]:
# --- 3. 데이터 기본 전처리 ---
print("\n--- 3. 데이터 기본 전처리 ---")

# 사용자 설정 변수 정의 (코드 상단 또는 이 섹션 시작 부분에 위치)
YEAR_COLUMN = 'year'
TARGET_COLUMN = 'modularity_class'
ID_COLUMN = 'Id' # 노드 ID 컬럼 이름 정의

# 필요한 컬럼 존재 확인
required_cols_check = [YEAR_COLUMN, TARGET_COLUMN, ID_COLUMN]
if not all(col in df.columns for col in required_cols_check):
    print(f"오류: 필요한 컬럼({required_cols_check}) 중 일부가 df에 없습니다.")
    exit()

# year 컬럼 숫자형 변환 및 결측치 처리
df[YEAR_COLUMN] = pd.to_numeric(df[YEAR_COLUMN], errors='coerce')
df.dropna(subset=[YEAR_COLUMN], inplace=True)
df[YEAR_COLUMN] = df[YEAR_COLUMN].astype(int)

# target 컬럼 숫자형 변환 및 결측치 처리
df[TARGET_COLUMN] = pd.to_numeric(df[TARGET_COLUMN], errors='coerce').fillna(-1).astype(int)

# --- 노드 ID 컬럼을 인덱스로 설정 (한 번만 실행) ---
print(f"현재 df 인덱스 타입: {type(df.index)}, 인덱스 이름: {df.index.name}")
if isinstance(df.index, pd.RangeIndex) and ID_COLUMN in df.columns:
     df.set_index(ID_COLUMN, inplace=True) # 'Id' 컬럼을 인덱스로 설정!
     print(f"'{ID_COLUMN}' 컬럼을 인덱스로 설정했습니다. 새 인덱스 타입: {df.index.dtype}")
elif df.index.name == ID_COLUMN:
     print(f"'{ID_COLUMN}'이 이미 인덱스로 설정되어 있습니다.")
else:
     # RangeIndex도 아니고, 이름이 ID_COLUMN인 인덱스도 아닌 경우
     # -> ID_COLUMN이 존재하면 인덱스로 설정 시도, 없으면 오류
     if ID_COLUMN in df.columns:
         try:
             df.set_index(ID_COLUMN, inplace=True)
             print(f"'{ID_COLUMN}' 컬럼을 인덱스로 설정했습니다. 새 인덱스 타입: {df.index.dtype}")
         except KeyError:
              print(f"오류: '{ID_COLUMN}' 컬럼을 인덱스로 설정하는 데 실패했습니다.")
              exit()
     else:
         print(f"오류: '{ID_COLUMN}' 컬럼이 df에 없고, 현재 인덱스도 '{ID_COLUMN}'이 아닙니다.")
         exit() # ID 컬럼 없으면 중단
# --- 인덱스 설정 끝 ---

print("데이터 기본 전처리 완료.")


--- 3. 데이터 기본 전처리 ---
현재 df 인덱스 타입: <class 'pandas.core.indexes.base.Index'>, 인덱스 이름: None
'Id' 컬럼을 인덱스로 설정했습니다. 새 인덱스 타입: object
데이터 기본 전처리 완료.


In [11]:
# --- 4. 텍스트 특징 생성 (TF-IDF) ---
print("\n--- 4. 텍스트 특징 생성 ---")
corpus = []
actual_text_cols = [col for col in TEXT_FEATURE_COLS if col in df.columns] # 실제 존재하는 컬럼만 사용
print(f"사용할 텍스트 특징 컬럼: {actual_text_cols}")

# (V3 Corpus 생성 루프는 이전과 동일하게 사용)
for index, row in df.iterrows(): # index는 'Id'
    text_parts = []
    for col in actual_text_cols:
        cell_value = row[col]
        try:
            if isinstance(cell_value, str):
                cleaned_value = cell_value.strip()
                if cleaned_value: text_parts.append(cleaned_value)
            elif isinstance(cell_value, (list, tuple, np.ndarray, pd.Series)):
                list_parts = []
                for item in cell_value:
                    if pd.notna(item) and isinstance(item, str):
                        cleaned_item = item.strip()
                        if cleaned_item: list_parts.append(cleaned_item)
                if list_parts: text_parts.append(" ".join(list_parts))
            elif pd.notna(cell_value):
                cleaned_value = str(cell_value).strip()
                if cleaned_value: text_parts.append(cleaned_value)
        except Exception as e:
             print(f"  - Warning: Error processing cell - Index: {index}, Column: {col}, Value Type: {type(cell_value)}, Error: {e}")
             pass
    corpus.append(" ".join(text_parts))

print(f"Corpus 생성 완료 (총 {len(corpus)}개 문서).")


if corpus:
    # **** max_features 값을 대폭 줄임 (예: 100) ****
    vectorizer = TfidfVectorizer(max_features=100)
    tfidf_features_sparse = vectorizer.fit_transform(corpus)
    print(f"TF-IDF 특징 생성 완료 (희소 행렬). Shape: {tfidf_features_sparse.shape}")

    # **** .toarray() 다시 사용! ****
    try:
        node_features = tfidf_features_sparse.toarray()
        print(f"밀집 배열로 변환 완료. Shape: {node_features.shape}")
    except MemoryError:
         print("오류: max_features를 줄였음에도 MemoryError 발생! 더 줄이거나 다른 방법 필요.")
         exit()
    except Exception as e:
         print(f"밀집 배열 변환 중 오류: {e}")
         exit()
else:
    # **** corpus가 비어있을 경우 처리 ****
    print("경고: 텍스트 특징으로 사용할 Corpus가 비어있습니다.")
    # 이 경우 모델 학습에 특징을 사용할 수 없으므로 처리가 필요합니다.
    # 옵션 1: 모든 특징을 0으로 채운 임시 특징 생성 (차원 수는 맞춰야 함)
    # num_features = 100 # max_features와 동일하게 설정
    # if 'df' in locals() and isinstance(df, pd.DataFrame):
    #      node_features = np.zeros((len(df), num_features))
    #      print(f"임시 특징(0) 생성 완료. Shape: {node_features.shape}")
    # else:
    #      print("오류: df 변수를 찾을 수 없어 임시 특징을 생성할 수 없습니다.")
    #      node_features = None # node_features를 None으로 유지

    # 옵션 2: 특징 없이 진행하도록 None 유지 (이후 코드에서 None 처리 필요)
    node_features = None
    print("node_features를 None으로 설정합니다.")

# --- 최종 노드 특징 준비 ---
if node_features is not None: # <--- 기준 라인
    # **** 아래 코드 블록 전체가 들여쓰기 되어야 함 ****
    try: # <--- if 보다 들여쓰기 됨
        # df의 인덱스 상태 확인 및 node_feature_df 생성
        if 'Id' in df.columns and isinstance(df.index, pd.RangeIndex): # <--- try 보다 들여쓰기 됨
             node_feature_df = pd.DataFrame(node_features, index=df['Id'])
             df.set_index('Id', inplace=True)
        elif df.index.name == 'Id': # <--- try 보다 들여쓰기 됨
             node_feature_df = pd.DataFrame(node_features, index=df.index)
        # ... (이하 try 블록 내 다른 코드들도 동일하게 들여쓰기) ...

        print(f"최종 노드 특징 (밀집 배열) 준비 완료. Shape: {node_feature_df.shape}")

    except Exception as e: # <--- try 와 같은 수준으로 들여쓰기 됨
        print(f"노드 특징 DataFrame 생성 중 오류: {e}") # <--- except 보다 들여쓰기 됨
        exit()
else: # <--- if 와 같은 수준으로 들여쓰기 됨
    print("오류: node_features가 None입니다.") # <--- else 보다 들여쓰기 됨
    exit()


--- 4. 텍스트 특징 생성 ---
사용할 텍스트 특징 컬럼: ['keywords', 'research_fields', 'Label']
Corpus 생성 완료 (총 5259857개 문서).
TF-IDF 특징 생성 완료 (희소 행렬). Shape: (5259857, 100)
밀집 배열로 변환 완료. Shape: (5259857, 100)
최종 노드 특징 (밀집 배열) 준비 완료. Shape: (5259857, 100)


In [12]:
# --- 5. 훈련/예측 대상 노드 ID 분리 ---
print("\n--- 5. 훈련/예측 대상 노드 ID 분리 ---")
period_6_end_year = 2019
period_7_start_year = 2020
period_7_end_year = 2022

# df의 인덱스('Id')를 기준으로 필터링
train_node_ids = df[df[YEAR_COLUMN] <= period_6_end_year].index
predict_node_ids = df[(df[YEAR_COLUMN] >= period_7_start_year) & (df[YEAR_COLUMN] <= period_7_end_year)].index
print(f"훈련 노드 수 (Period 1-6): {len(train_node_ids)}")
print(f"예측 대상 노드 수 (Period 7): {len(predict_node_ids)}")


--- 5. 훈련/예측 대상 노드 ID 분리 ---
훈련 노드 수 (Period 1-6): 4420308
예측 대상 노드 수 (Period 7): 837944


In [13]:
# --- 6. 훈련 레이블 준비 (One-Hot Encoding) ---
print("\n--- 6. 훈련 레이블 준비 ---")
if len(train_node_ids) > 0:
    train_labels_raw = df.loc[train_node_ids, TARGET_COLUMN]
    if train_labels_raw.nunique() <= 1:
         print("오류: 훈련 데이터에 레이블 종류가 1개 이하입니다. 분류 모델 학습 불가.")
         exit()

    target_encoder = LabelBinarizer()
    train_targets = target_encoder.fit_transform(train_labels_raw)
    num_classes = train_targets.shape[1]
    print(f"훈련 레이블 One-Hot 인코딩 완료. 클래스 수: {num_classes}")
    target_label_map = {i: label for i, label in enumerate(target_encoder.classes_)}
    print(f"클래스 매핑: {target_label_map}")
else:
    print("오류: 훈련 데이터가 없습니다.")
    # 필요한 변수들 초기화 또는 exit()
    train_targets = None
    target_encoder = None
    target_label_map = None
    num_classes = 0



--- 6. 훈련 레이블 준비 ---
훈련 레이블 One-Hot 인코딩 완료. 클래스 수: 62
클래스 매핑: {0: -1, 1: 0, 2: 1, 3: 2, 4: 3, 5: 4, 6: 5, 7: 6, 8: 7, 9: 8, 10: 9, 11: 10, 12: 11, 13: 12, 14: 13, 15: 14, 16: 15, 17: 16, 18: 17, 19: 18, 20: 19, 21: 20, 22: 21, 23: 22, 24: 23, 25: 24, 26: 25, 27: 26, 28: 27, 29: 28, 30: 29, 31: 30, 32: 31, 33: 32, 34: 33, 35: 34, 36: 35, 37: 36, 38: 37, 39: 38, 40: 39, 41: 40, 42: 41, 43: 42, 44: 43, 45: 44, 46: 45, 47: 46, 48: 47, 49: 48, 50: 49, 51: 50, 52: 51, 53: 52, 54: 53, 55: 54, 56: 55, 57: 56, 58: 57, 59: 58, 60: 60, 61: 61}


In [14]:
# --- 7. 엣지 데이터 필터링 --- (전에 실행하여 확인)

print("\n--- ID 일관성 확인 시작 ---")

# df의 인덱스 정보 확인
print(f"df 인덱스 이름: {df.index.name}")
print(f"df 인덱스 타입: {df.index.dtype}")
print(f"df 인덱스 샘플 (상위 5개): {df.index[:5].tolist()}") # 샘플 확인

# edges_df의 Source/Target 컬럼 정보 확인
if 'Source' in edges_df.columns and 'Target' in edges_df.columns:
    print(f"\nedges_df['Source'] 타입: {edges_df['Source'].dtype}")
    print(f"edges_df['Source'] 샘플 (상위 5개): {edges_df['Source'].head().tolist()}")
    print(f"\nedges_df['Target'] 타입: {edges_df['Target'].dtype}")
    print(f"edges_df['Target'] 샘플 (상위 5개): {edges_df['Target'].head().tolist()}")

    # 샘플 ID 직접 비교 (첫 번째 엣지의 Source ID가 df 인덱스에 있는지 확인)
    if not edges_df.empty:
        sample_edge_source_id = edges_df['Source'].iloc[0]
        # 타입을 문자열로 통일하여 비교 준비
        sample_edge_source_id_str = str(sample_edge_source_id)
        df_index_str_set = set(df.index.astype(str)) # df 인덱스도 문자열 Set으로

        print(f"\n첫 번째 엣지의 Source ID: '{sample_edge_source_id}' (타입: {type(sample_edge_source_id)})")
        if sample_edge_source_id_str in df_index_str_set:
            print(f"-> 이 ID는 df 인덱스에 존재합니다.")
        else:
            print(f"-> **** 이 ID가 df 인덱스에 존재하지 않습니다! ****")
            # 존재하지 않는 이유 추적: 공백 문제? 대소문자 문제? 아예 없는 ID?
            # 예: print(f"   '{sample_edge_source_id_str.strip()}' in df_index_str_set ? {sample_edge_source_id_str.strip() in df_index_str_set}")
            # 예: print(f"   '{sample_edge_source_id_str.lower()}' in set(idx.lower() for idx in df_index_str_set) ? {sample_edge_source_id_str.lower() in set(idx.lower() for idx in df_index_str_set)}")

else:
    print("\n오류: edges_df에 'Source' 또는 'Target' 컬럼이 없습니다.")

print("--- ID 일관성 확인 끝 ---")



# --- 7. 엣지 데이터 필터링 ---
print("\n--- 7. 엣지 데이터 필터링 ---")
valid_node_ids = set(df.index.astype(str)) # 모든 노드 ID (문자열로 변환하여 비교)
original_edge_count = len(edges_df)

# Source와 Target 노드가 모두 valid_node_ids에 존재하는 엣지만 남김 (타입 통일)
edges_df_filtered = edges_df[
    edges_df['Source'].astype(str).isin(valid_node_ids) &
    edges_df['Target'].astype(str).isin(valid_node_ids)
].copy()

filtered_edge_count = len(edges_df_filtered)
print(f"필터링 후 엣지 수: {filtered_edge_count} (원본: {original_edge_count}, 제거됨: {original_edge_count - filtered_edge_count})")
print("\n--- 데이터 준비 완료 ---")

# 이제 'node_feature_df', 'edges_df_filtered', 'train_node_ids',
# 'predict_node_ids', 'train_targets', 'target_encoder', 'target_label_map', 'num_classes'
# 변수들을 사용하여 StellarGraph 객체 생성 및 모델 빌드/학습을 진행할 수 있습니다.


--- ID 일관성 확인 시작 ---
df 인덱스 이름: Id
df 인덱스 타입: object
df 인덱스 샘플 (상위 5개): ['53a725db20f7420be8b5bfc6', '53a725db20f7420be8b5bff3', '53a725db20f7420be8b5bffb', '53a725db20f7420be8b5c00f', '53a727dc20f7420be8b9eed7']

edges_df['Source'] 타입: object
edges_df['Source'] 샘플 (상위 5개): ['53e9979bb7602d9701f66d44', '53e9979bb7602d9701f66d44', '53e9979bb7602d9701f66d44', '53e9979bb7602d9701f66d44', '53e9979bb7602d9701f66d44']

edges_df['Target'] 타입: object
edges_df['Target'] 샘플 (상위 5개): ['53e9979bb7602d9701f66d44', '53e9981db7602d970203b7b2', '53e99a98b7602d97023101c5', '53e9ab3eb7602d97034d0b36', '53e9ab69b7602d9703504883']

첫 번째 엣지의 Source ID: '53e9979bb7602d9701f66d44' (타입: <class 'str'>)
-> 이 ID는 df 인덱스에 존재합니다.
--- ID 일관성 확인 끝 ---

--- 7. 엣지 데이터 필터링 ---
필터링 후 엣지 수: 1391357 (원본: 1391367, 제거됨: 10)

--- 데이터 준비 완료 ---


In [15]:
# --- 필요한 라이브러리 임포트 (이미 임포트했다면 생략 가능) ---
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelBinarizer

try:
    import stellargraph as sg
    from stellargraph.mapper import GraphSAGENodeGenerator
    from stellargraph.layer import GraphSAGE
    print(f"StellarGraph version: {sg.__version__}")
except ImportError:
    print("StellarGraph를 찾을 수 없습니다. 설치를 확인하세요.")
    exit()

try:
    import tensorflow as tf
    from tensorflow.keras import layers, optimizers, losses, metrics, Model
    print(f"TensorFlow version: {tf.__version__}")
except ImportError:
    print("TensorFlow를 찾을 수 없습니다. 설치를 확인하세요.")
    exit()

StellarGraph version: 1.2.1
TensorFlow version: 2.9.1


In [16]:
# --- 이전 단계에서 생성된 변수 확인 (예시) ---
# 이 변수들이 현재 메모리에 존재해야 합니다.
print(f"\n데이터 준비 결과 확인:")
print(f"  - node_feature_df shape: {node_feature_df.shape if 'node_feature_df' in locals() else 'Not Found'}")
print(f"  - edges_df_filtered count: {len(edges_df_filtered) if 'edges_df_filtered' in locals() else 'Not Found'}")
print(f"  - train_node_ids count: {len(train_node_ids) if 'train_node_ids' in locals() else 'Not Found'}")
print(f"  - predict_node_ids count: {len(predict_node_ids) if 'predict_node_ids' in locals() else 'Not Found'}")
print(f"  - train_targets shape: {train_targets.shape if 'train_targets' in locals() else 'Not Found'}")
print(f"  - num_classes: {num_classes if 'num_classes' in locals() else 'Not Found'}")



데이터 준비 결과 확인:
  - node_feature_df shape: (5259857, 100)
  - edges_df_filtered count: 1391357
  - train_node_ids count: 4420308
  - predict_node_ids count: 837944
  - train_targets shape: (4420308, 62)
  - num_classes: 62


In [17]:
# --- 2. StellarGraph 객체 생성 ---
print("\n--- 2. StellarGraph 객체 생성 ---")
try:
    # 노드 ID와 특징 인덱스 일치 확인 (이전 코드와 동일)
    if 'df' in locals() and 'node_feature_df' in locals() and df.index.equals(node_feature_df.index):
        print("노드 ID와 특징 인덱스 일치 확인됨.")
    else:
        print("오류: df 또는 node_feature_df 변수를 찾을 수 없거나 인덱스가 일치하지 않습니다.")
        exit()

    # **** 수정: StellarGraph에 필요한 엣지 컬럼만 선택 ****
    if 'edges_df_filtered' in locals() and isinstance(edges_df_filtered, pd.DataFrame):
        # 'Source'와 'Target' 컬럼만 선택하여 새로운 DataFrame 생성
        edges_for_graph = edges_df_filtered[['Source', 'Target']].copy()
        print(f"StellarGraph 생성을 위해 엣지 {len(edges_for_graph)}개 준비 (Source, Target 컬럼만 사용).")
    else:
        print("오류: 필터링된 엣지 데이터('edges_df_filtered')를 찾을 수 없습니다.")
        exit()
    # ****************************************************

    # 수정된 edges_for_graph 사용
    G = sg.StellarGraph(nodes=node_feature_df, edges=edges_for_graph, source_column='Source', target_column='Target')

    print("StellarGraph 객체 생성 완료.")
    print(G.info())

except KeyError as e:
    print(f"StellarGraph 객체 생성 중 오류: 엣지 데이터프레임({e}) 또는 노드 데이터에 필요한 컬럼이 없습니다.")
    # print(f"  엣지 컬럼: {edges_for_graph.columns.tolist() if 'edges_for_graph' in locals() else 'N/A'}")
    # print(f"  노드 특징 컬럼: {node_feature_df.columns.tolist() if 'node_feature_df' in locals() else 'N/A'}")
    exit()
except Exception as e:
    print(f"StellarGraph 객체 생성 중 오류: {e}")
    exit()

# (이하 모델 빌드 코드 계속)


--- 2. StellarGraph 객체 생성 ---
노드 ID와 특징 인덱스 일치 확인됨.
StellarGraph 생성을 위해 엣지 1391357개 준비 (Source, Target 컬럼만 사용).
StellarGraph 객체 생성 완료.
StellarGraph: Undirected multigraph
 Nodes: 5259857, Edges: 1391357

 Node types:
  default: [5259857]
    Features: float32 vector, length 100
    Edge types: default-default->default

 Edge types:
    default-default->default: [1391357]
        Weights: all 1 (default)
        Features: none


In [18]:



# --- 3. GraphSAGE 모델 빌드 ---
print("\n--- 3. GraphSAGE 모델 빌드 ---")
if G is not None and 'train_node_ids' in locals() and len(train_node_ids) > 0 and 'train_targets' in locals():
    # 하이퍼파라미터 (조절 필요)
    batch_size = 50       # 메모리 상황에 따라 조절 (더 작게 시도 가능)
    num_samples = [10, 5] # 샘플링할 이웃 수 (메모리/시간 영향 큼)
    epochs = 5           # **주의: 초기 테스트를 위해 에폭 수를 매우 작게 설정 (실제로는 더 많이 필요)**
    learning_rate = 1e-3
    layer_sizes = [32, 32] # 레이어 유닛 수 (메모리 영향)

    # 3.1. 데이터 제너레이터 생성
    generator = GraphSAGENodeGenerator(G, batch_size, num_samples)

    # 3.2. 훈련/검증 데이터 분할 및 flow 생성
    # **** 중요: -1 레이블 제외 로직 적용된 변수 사용 ****
    # 이전 단계에서 -1을 제외한 train_node_ids_filtered 와 train_labels_raw 를 사용해야 함
    # (만약 이전 단계에서 해당 변수명을 사용했다면 아래 코드 수정 필요)

    # 여기서는 -1 레이블 제외 코드가 바로 이전 단계에 없었으므로, 다시 적용합니다.
    # (만약 이전 '6. 훈련 레이블 준비' 단계에서 이미 -1을 제외했다면 이 부분은 생략하고
    #  train_test_split에 필터링된 ID와 레이블을 직접 사용해야 합니다.)
    if 'train_labels_raw' in locals() and 'target_encoder' in locals():
        valid_train_mask = (train_labels_raw != -1)
        train_ids_for_split = train_node_ids[valid_train_mask]
        train_targets_for_split = target_encoder.transform(train_labels_raw[valid_train_mask])
        train_labels_raw_for_stratify = train_labels_raw[valid_train_mask] # stratify 용도

        if len(train_ids_for_split) > 1: # 분할 가능한지 확인
            print(f"훈련/검증 분할 시작 (stratify 없이 무작위 분할). 총 {len(train_ids_for_split)}개 샘플.")
            # **** stratify 파라미터 제거 ****
            train_set_ids, val_set_ids = train_test_split(
                train_ids_for_split,  # -1 제외된 유효 훈련 노드 ID
                test_size=0.1,        # 검증 세트 비율 (10%)
                # stratify=train_labels_raw_for_stratify, # <-- 이 줄 제거 또는 주석 처리
                random_state=42       # <-- 재현성을 위해 추가하는 것이 좋음 (선택 사항)
            )

            # **** 중요: 이후 타겟 생성 시에도 필터링된 ID 사용 ****
            train_targets_split = target_encoder.transform(df.loc[train_set_ids, TARGET_COLUMN])
            val_targets_split = target_encoder.transform(df.loc[val_set_ids, TARGET_COLUMN])

            train_gen = generator.flow(train_set_ids, train_targets_split, shuffle=True)
            val_gen = generator.flow(val_set_ids, val_targets_split)
            print(f"훈련/검증 데이터 생성 완료. 훈련 샘플: {len(train_set_ids)}, 검증 샘플: {len(val_set_ids)}")
        else:
            print("오류: 유효한 훈련 데이터가 부족하여 분할/학습할 수 없습니다.")
            train_gen = None
            val_gen = None
    else:
        print("오류: 훈련 레이블 정보가 없어 제너레이터를 생성할 수 없습니다.")
        train_gen = None
        val_gen = None


    # 3.4. Keras 모델 정의 (train_gen이 생성된 경우에만)
    if train_gen is not None:
        graphsage = GraphSAGE(
            layer_sizes=layer_sizes, generator=generator, bias=True, dropout=0.3
        )
        x_inp, x_out = graphsage.in_out_tensors()
        prediction = layers.Dense(units=num_classes, activation="softmax")(x_out) # num_classes는 이전 단계에서 계산됨

        model = Model(inputs=x_inp, outputs=prediction)

        # 3.5. 모델 컴파일
        model.compile(
            optimizer=optimizers.Adam(learning_rate=learning_rate),
            loss=losses.categorical_crossentropy,
            metrics=[metrics.categorical_accuracy],
        )
        print("\n--- 모델 빌드 완료 ---")
        print(model.summary()) # 모델 구조 출력


        # --- 4. 모델 학습 ---
        print("\n--- 4. 모델 학습 ---")
        # 학습 데이터가 있고, 모델이 정의된 경우에만 학습 진행
        history = model.fit(
            train_gen,
            epochs=epochs,          # **주의: 테스트를 위해 에폭 수를 작게 설정 (5)**
            validation_data=val_gen,
            verbose=1,              # 진행 상황을 더 자세히 보기 위해 1로 변경
            shuffle=False
        )
        print("\n--- 모델 학습 완료 ---")
        # (선택 사항) 학습 곡선 시각화
        # sg.utils.plot_history(history)
        # plt.show()


        # --- 5. 기간 7 노드 예측 ---
        print("\n--- 5. 기간 7 노드 예측 ---")
        if 'predict_node_ids' in locals() and len(predict_node_ids) > 0 :
            predict_gen = generator.flow(predict_node_ids)
            p7_preds_probs = model.predict(predict_gen)

            p7_pred_indices = np.argmax(p7_preds_probs, axis=1)
            # target_label_map은 이전 단계에서 생성됨
            p7_pred_labels = [target_label_map.get(idx, -99) for idx in p7_pred_indices]

            # 예측 결과 저장 (df는 Id를 인덱스로 가짐)
            df.loc[predict_node_ids, f'{TARGET_COLUMN}_GraphSAGE'] = p7_pred_labels

            print(f"기간 7 노드 {len(predict_node_ids)}개에 대한 예측 완료.")
            print("예측 결과 샘플 (Modularity Class / GraphSAGE 예측):")
            # 인덱스를 사용하여 샘플 출력
            print(df.loc[predict_node_ids[:5], [TARGET_COLUMN, f'{TARGET_COLUMN}_GraphSAGE']])

            # (선택 사항) 전체 df를 결과 포함하여 저장
            # df.reset_index().to_csv('nodes_with_predictions.csv', index=False, encoding='utf-8-sig')

        else:
            print("정보: 예측할 기간 7 노드가 없습니다.")

    else:
        print("\n오류: 모델 학습에 필요한 데이터 제너레이터를 생성하지 못했습니다.")

else:
    print("\n오류: 모델 빌드 및 학습에 필요한 데이터/변수가 준비되지 않았습니다.")


--- 3. GraphSAGE 모델 빌드 ---
훈련/검증 분할 시작 (stratify 없이 무작위 분할). 총 336199개 샘플.
훈련/검증 데이터 생성 완료. 훈련 샘플: 302579, 검증 샘플: 33620

--- 모델 빌드 완료 ---
Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_2 (InputLayer)           [(None, 10, 100)]    0           []                               
                                                                                                  
 input_3 (InputLayer)           [(None, 50, 100)]    0           []                               
                                                                                                  
 input_1 (InputLayer)           [(None, 1, 100)]     0           []                               
                                                                                                  
 reshape (Reshape)              (None, 1, 10, 100)   0

--- 3. GraphSAGE 모델 빌드 ---
훈련/검증 분할 시작 (stratify 없이 무작위 분할). 총 336199개 샘플.
훈련/검증 데이터 생성 완료. 훈련 샘플: 302579, 검증 샘플: 33620

--- 모델 빌드 완료 ---
Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
==================================================================================================
 input_2 (InputLayer)           [(None, 10, 100)]    0           []                               
                                                                                                  
 input_3 (InputLayer)           [(None, 50, 100)]    0           []                               
                                                                                                  
 input_1 (InputLayer)           [(None, 1, 100)]     0           []                               
                                                                                                  
 reshape (Reshape)              (None, 1, 10, 100)   0           ['input_2[0][0]']                
                                                                                                  
 reshape_1 (Reshape)            (None, 10, 5, 100)   0           ['input_3[0][0]']                
                                                                                                  
 dropout_1 (Dropout)            (None, 1, 100)       0           ['input_1[0][0]']                
                                                                                                  
 dropout (Dropout)              (None, 1, 10, 100)   0           ['reshape[0][0]']                
                                                                                                  
 dropout_3 (Dropout)            (None, 10, 100)      0           ['input_2[0][0]']                
                                                                                                  
 dropout_2 (Dropout)            (None, 10, 5, 100)   0           ['reshape_1[0][0]']              
                                                                                                  
 mean_aggregator (MeanAggregato  multiple            3232        ['dropout_1[0][0]',              
 r)                                                               'dropout[0][0]',                
                                                                  'dropout_3[0][0]',              
                                                                  'dropout_2[0][0]']              
                                                                                                  
 reshape_2 (Reshape)            (None, 1, 10, 32)    0           ['mean_aggregator[1][0]']        
                                                                                                  
 dropout_5 (Dropout)            (None, 1, 32)        0           ['mean_aggregator[0][0]']        
                                                                                                  
 dropout_4 (Dropout)            (None, 1, 10, 32)    0           ['reshape_2[0][0]']              
                                                                                                  
 mean_aggregator_1 (MeanAggrega  (None, 1, 32)       1056        ['dropout_5[0][0]',              
 tor)                                                             'dropout_4[0][0]']              
                                                                                                  
 reshape_3 (Reshape)            (None, 32)           0           ['mean_aggregator_1[0][0]']      
                                                                                                  
 lambda (Lambda)                (None, 32)           0           ['reshape_3[0][0]']              
                                                                                                  
 dense (Dense)                  (None, 62)           2046        ['lambda[0][0]']                 
                                                                                                  
==================================================================================================
Total params: 6,334
Trainable params: 6,334
Non-trainable params: 0
__________________________________________________________________________________________________
None

--- 4. 모델 학습 ---
Epoch 1/5
6052/6052 [==============================] - 165s 27ms/step - loss: 1.7369 - categorical_accuracy: 0.5303 - val_loss: 1.3089 - val_categorical_accuracy: 0.6189
Epoch 2/5
6052/6052 [==============================] - 169s 28ms/step - loss: 1.3368 - categorical_accuracy: 0.6107 - val_loss: 1.2238 - val_categorical_accuracy: 0.6419
Epoch 3/5
6052/6052 [==============================] - 164s 27ms/step - loss: 1.2901 - categorical_accuracy: 0.6214 - val_loss: 1.1891 - val_categorical_accuracy: 0.6497
Epoch 4/5
6052/6052 [==============================] - 152s 25ms/step - loss: 1.2682 - categorical_accuracy: 0.6276 - val_loss: 1.1722 - val_categorical_accuracy: 0.6557
Epoch 5/5
6052/6052 [==============================] - 160s 26ms/step - loss: 1.2526 - categorical_accuracy: 0.6311 - val_loss: 1.1517 - val_categorical_accuracy: 0.6589

--- 모델 학습 완료 ---

--- 5. 기간 7 노드 예측 ---
16759/16759 [==============================] - 178s 11ms/step
기간 7 노드 837944개에 대한 예측 완료.
예측 결과 샘플 (Modularity Class / GraphSAGE 예측):
                          modularity_class  modularity_class_GraphSAGE
Id                                                                    
53e99785b7602d9701f402db                -1                        44.0
53e999fab7602d9702245390                -1                        44.0
53e999ffb7602d9702248487                -1                        44.0
53e99df7b7602d97026ba5aa                -1                        44.0
53e9a20fb7602d9702b129f1                -1                        44.0

In [19]:
import numpy as np
import pandas as pd # 혹시 모르니 임포트 확인

# --- 5. 기간 7 노드 예측 ---
print("\n--- 5. 기간 7 노드 예측 ---")

# 필요한 변수들이 존재하는지 확인 (더욱 안전하게)
required_vars_for_predict = ['model', 'generator', 'predict_node_ids', 'target_label_map', 'df', 'TARGET_COLUMN']
if not all(var in locals() or var in globals() for var in required_vars_for_predict):
    print("오류: 예측에 필요한 변수 중 일부가 정의되지 않았습니다.")
    print(f"필요 변수: {required_vars_for_predict}")
    # 필요한 경우 여기서 중단 (exit() 등)

# 예측 대상 노드가 있고, 학습된 모델과 제너레이터가 있을 경우 진행
elif len(predict_node_ids) > 0 and 'model' in locals() and 'generator' in locals():
    try:
        # 5.1. 예측용 데이터 제너레이터 생성
        #      predict_node_ids의 ID들이 generator 생성 시 사용된 G 객체에 있는지 확인 필요
        #      (일반적으로 df에서 추출했으므로 문제 없음)
        print(f"기간 7 노드 {len(predict_node_ids)}개에 대한 예측 제너레이터 생성 중...")
        predict_gen = generator.flow(predict_node_ids)
        print("예측 제너레이터 생성 완료.")

        # 5.2. 모델 예측 수행 -> 클래스별 확률 얻기
        print("모델 예측 수행 중 (시간이 걸릴 수 있습니다)...")
        p7_preds_probs = model.predict(predict_gen)
        print("모델 예측 완료.")

        # 5.3. 가장 높은 확률의 클래스 인덱스 찾기
        p7_pred_indices = np.argmax(p7_preds_probs, axis=1)

        # 5.4. 클래스 인덱스를 원래 Modularity Class 레이블로 변환
        #      target_label_map은 이전 '6. 훈련 레이블 준비' 단계에서 생성됨
        p7_pred_labels = [target_label_map.get(idx, -99) for idx in p7_pred_indices] # 매핑 실패 시 -99

        # 5.5. 예측 결과 저장 (df는 Id를 인덱스로 가지고 있어야 함)
        predict_col_name = f'{TARGET_COLUMN}_GraphSAGE'
        print(f"예측 결과를 '{predict_col_name}' 컬럼에 저장 중...")
        if df.index.name == 'Id': # df의 인덱스가 'Id'인지 재확인
             # predict_node_ids 와 p7_pred_labels 길이가 같은지 확인 (디버깅용)
             if len(predict_node_ids) == len(p7_pred_labels):
                  df.loc[predict_node_ids, predict_col_name] = p7_pred_labels
                  print(f"기간 7 노드 {len(predict_node_ids)}개에 대한 예측 완료.")
                  print("예측 결과 샘플 (원본 Modularity Class / GraphSAGE 예측):")
                  # 인덱스를 사용하여 샘플 출력
                  print(df.loc[predict_node_ids[:5], [TARGET_COLUMN, predict_col_name]])
             else:
                  print(f"오류: 예측 대상 ID 수({len(predict_node_ids)})와 예측된 레이블 수({len(p7_pred_labels)})가 다릅니다!")
        else:
             print("오류: 예측 결과를 저장하기 위해 df의 인덱스가 'Id'여야 합니다.")


        # (선택 사항) 전체 df를 결과 포함하여 저장
        # try:
        #     save_path = 'nodes_with_predictions.csv'
        #     df.reset_index().to_csv(save_path, index=False, encoding='utf-8-sig')
        #     print(f"예측 결과 포함된 전체 노드 데이터 저장 완료: {save_path}")
        # except Exception as e_save:
        #      print(f"결과 저장 중 오류 발생: {e_save}")

    except Exception as e:
        print(f"예측 중 오류 발생: {e}")

elif len(predict_node_ids) == 0:
    print("정보: 예측할 기간 7 노드가 없습니다.")
else:
    print("오류: 모델이 학습되지 않았거나 필요한 변수가 없어 예측을 수행할 수 없습니다.")


--- 5. 기간 7 노드 예측 ---
기간 7 노드 837944개에 대한 예측 제너레이터 생성 중...
예측 제너레이터 생성 완료.
모델 예측 수행 중 (시간이 걸릴 수 있습니다)...
16759/16759 [==============================] - 166s 10ms/step
모델 예측 완료.
예측 결과를 'modularity_class_GraphSAGE' 컬럼에 저장 중...
기간 7 노드 837944개에 대한 예측 완료.
예측 결과 샘플 (원본 Modularity Class / GraphSAGE 예측):
                          modularity_class  modularity_class_GraphSAGE
Id                                                                    
53e99785b7602d9701f402db                -1                        44.0
53e999fab7602d9702245390                -1                        44.0
53e999ffb7602d9702248487                -1                        44.0
53e99df7b7602d97026ba5aa                -1                        44.0
53e9a20fb7602d9702b129f1                -1                        44.0


In [20]:
df.head()

,Degree,Label,citations,indegree,keywords,level,modularity_class,outdegree,research_fields,venue,year,modularity_class_GraphSAGE
Id,,,,,,,,,,,,
53a725db20f7420be8b5bfc6,NaN,Deductive Algorithmic Knowledge,37.0,NaN,"[deductive system, fixed deductive system, pro...",NaN,-1,NaN,[],Journal of Logic and Computation,2004,NaN
53a725db20f7420be8b5bff3,NaN,Improving exact algorithms for MAX-2-SAT,26.0,NaN,"[new variable, clause-to-variable ratio, graph...",NaN,-1,NaN,[],Annals of Mathematics and Artificial Intelligence,2004,NaN
53a725db20f7420be8b5bffb,NaN,Learning via finitely many queries,0.0,NaN,"[new query inference model, Boolean query, exp...",NaN,-1,NaN,[],Annals of Mathematics and Artificial Intelligence,2004,NaN
53a725db20f7420be8b5c00f,NaN,A framework for sequential planning in multi-a...,501.0,NaN,"[belief state, value function, approximate bel...",NaN,-1,NaN,"[Mathematical optimization, Partially observab...",Journal of Artificial Intelligence Research,2004,NaN
53a727dc20f7420be8b9eed7,NaN,Evaluation of Prediction Methods Applied to an...,5.0,NaN,"[Basurde Spanish dialogue system, dialogue sta...",NaN,-1,NaN,"[Morpheme, Hit ratio, Computer science, Keystr...",TSD '02 Proceedings of the 5th International C...,2002,NaN


In [22]:
import pandas as pd
import os
import numpy as np # 혹시 모를 numpy 사용을 위해

# --- 사용자 설정: 저장 경로 및 파일 이름 ---
# 저장할 디렉토리를 지정합니다.
output_dir_final = "C:/Project/RAG연구동향분석/250409/Final_Results"
# 전체 데이터 파일 이름
full_df_filename = "nodes_with_predictions_full.csv"
# 기간 7 데이터 파일 이름
period7_df_filename = "nodes_period7_predictions.csv"

output_path_full = os.path.join(output_dir_final, full_df_filename)
output_path_p7 = os.path.join(output_dir_final, period7_df_filename)

# 저장할 디렉토리가 없으면 생성
os.makedirs(output_dir_final, exist_ok=True)

# 필요한 컬럼 이름 확인
YEAR_COLUMN = 'year' # 실제 연도 컬럼 이름 확인
# -------------------------------------------

# --- 1. 전체 df 저장 ---
print(f"\n--- 1. 전체 데이터프레임 저장 ---")
if 'df' in locals() and isinstance(df, pd.DataFrame):
    try:
        print(f"'{output_path_full}' 경로에 전체 노드 데이터(예측 포함) 저장 시작...")
        # 'Id' 인덱스를 컬럼으로 변환 후 저장, 기본 RangeIndex는 저장 안 함
        df.reset_index().to_csv(output_path_full, index=False, encoding='utf-8-sig')
        print(f"전체 데이터 저장 완료: {output_path_full}")
    except Exception as e:
        print(f"전체 데이터 저장 중 오류 발생: {e}")
else:
    print("오류: 저장할 'df' 데이터프레임을 찾을 수 없습니다.")


# --- 2. 기간 7 데이터 필터링 및 저장 ---
print(f"\n--- 2. 기간 7 데이터 필터링 및 저장 ---")
if 'df' in locals() and isinstance(df, pd.DataFrame) and YEAR_COLUMN in df.columns:
    try:
        # 기간 7 정의
        period7_start = 2020
        period7_end = 2022

        # 기간 7 데이터 필터링
        print(f"'{YEAR_COLUMN}' 컬럼 기준으로 {period7_start}~{period7_end} 데이터 필터링 중...")
        # df의 year 컬럼 타입을 다시 확인 (정수형이어야 함)
        if not pd.api.types.is_integer_dtype(df[YEAR_COLUMN]):
             print(f"  경고: '{YEAR_COLUMN}' 컬럼이 정수형이 아닙니다. 변환 시도...")
             # 이미 이전 단계에서 정수 변환 했으므로 보통 문제 없음
             df[YEAR_COLUMN] = pd.to_numeric(df[YEAR_COLUMN], errors='coerce').fillna(-1).astype(int)

        period7_mask = (df[YEAR_COLUMN] >= period7_start) & (df[YEAR_COLUMN] <= period7_end)
        df_period7 = df[period7_mask].copy() # .copy() 사용하여 SettingWithCopyWarning 방지

        print(f"기간 7 데이터 {len(df_period7)}개 필터링 완료.")

        if not df_period7.empty:
            # 기간 7 데이터 저장
            print(f"'{output_path_p7}' 경로에 기간 7 데이터 저장 시작...")
            # 'Id' 인덱스를 컬럼으로 변환 후 저장
            df_period7.reset_index().to_csv(output_path_p7, index=False, encoding='utf-8-sig')
            print(f"기간 7 데이터 저장 완료: {output_path_p7}")
        else:
            print("정보: 기간 7에 해당하는 데이터가 없어 파일을 저장하지 않습니다.")

    except KeyError:
         print(f"오류: '{YEAR_COLUMN}' 컬럼이 df에 없습니다.")
    except Exception as e:
        print(f"기간 7 데이터 처리 또는 저장 중 오류 발생: {e}")

elif not ('df' in locals() and isinstance(df, pd.DataFrame)):
     print("오류: 'df' 데이터프레임을 찾을 수 없어 기간 7 데이터를 처리할 수 없습니다.")
else:
     print(f"오류: '{YEAR_COLUMN}' 컬럼이 df에 없어 기간 7 데이터를 처리할 수 없습니다.")


--- 1. 전체 데이터프레임 저장 ---
'C:/Project/RAG연구동향분석/250409/Final_Results\nodes_with_predictions_full.csv' 경로에 전체 노드 데이터(예측 포함) 저장 시작...
전체 데이터 저장 완료: C:/Project/RAG연구동향분석/250409/Final_Results\nodes_with_predictions_full.csv

--- 2. 기간 7 데이터 필터링 및 저장 ---
'year' 컬럼 기준으로 2020~2022 데이터 필터링 중...
기간 7 데이터 837944개 필터링 완료.
'C:/Project/RAG연구동향분석/250409/Final_Results\nodes_period7_predictions.csv' 경로에 기간 7 데이터 저장 시작...
기간 7 데이터 저장 완료: C:/Project/RAG연구동향분석/250409/Final_Results\nodes_period7_predictions.csv
